In [1]:
import torch
import numpy as np
import h5py
import sys
import psutil
process = psutil.Process()
print(process.memory_info().rss/1024/1024)  # used memory in mb
outfile="outs/testingmemleak.h5"
print(torch.__version__)
print(h5py.__version__)
# yay this has been fixed in a newer version of pytorch

274.0
1.12.1
3.7.0


In [2]:
fill_value = "value".encode('ascii')
large_array_bad = np.full((400,2000,8), fill_value)
# write a nice file
with h5py.File(outfile, 'w') as file:
    file.create_dataset('bad', data=large_array_bad)
print(process.memory_info().rss/1024/1024)

306.6875


In [3]:
for i in range(100):
    # read a nice file
    data_dict={}
    with h5py.File(outfile,'r') as file:
        for label, data in file.items():
            data_dict[label] = data[()]
    print(process.memory_info().rss/1024/1024)
    tensor_data = []
    for label, data in data_dict.items():  # try to convert to torch tensors
        try:
            tensor = torch.from_numpy(data)
            tensor_data.append(tensor)
        except TypeError:
            print("couldn't convert from numpy")
    print(process.memory_info().rss/1024/1024)  # the memory is increasing
    if process.memory_info().rss/1024/1024 > 1000:
        sys.exit("Too much memory used")

335.5625
couldn't convert from numpy
335.5625
368.5
couldn't convert from numpy
368.5
397.375
couldn't convert from numpy
397.375
426.25
couldn't convert from numpy
426.25
459.25
couldn't convert from numpy
459.25
488.125
couldn't convert from numpy
488.125
521.125
couldn't convert from numpy
521.125
550.0
couldn't convert from numpy
550.0
578.875
couldn't convert from numpy
578.875
611.875
couldn't convert from numpy
611.875
640.75
couldn't convert from numpy
640.75
673.75
couldn't convert from numpy
673.75
702.625
couldn't convert from numpy
702.625
735.625
couldn't convert from numpy
735.625
764.5
couldn't convert from numpy
764.5
793.375
couldn't convert from numpy
793.375
826.375
couldn't convert from numpy
826.375
855.25
couldn't convert from numpy
855.25
888.25
couldn't convert from numpy
888.25
917.125
couldn't convert from numpy
917.125
946.0
couldn't convert from numpy
946.0
979.0
couldn't convert from numpy
979.0
1007.875
couldn't convert from numpy
1007.875


SystemExit: Too much memory used

/home/kendrab/.conda/envs/torch-env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
